In [1]:
import random
import numpy as np
from imutils import paths
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
import cv2
import sys
import os
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
Y = os.listdir("save/")
Y.remove('.DS_Store')
label_encoder = LabelEncoder()
integer_result = label_encoder.fit(Y)
integer_result = label_encoder.transform(Y)
# One-Hot编码
one_hot_encoder = OneHotEncoder()
# One-Hot编码将分类值映射到整数值再表示成二进制向量
integer_result1 = integer_result.reshape(len(integer_result), 1)
#integer_result1 = integer_result1.flatten()
Y = one_hot_encoder.fit(integer_result1)

Y = one_hot_encoder.transform(integer_result1)

Y.toarray()

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [3]:
def generate_file_list(path,p):
    print("[INFO] loading images...")
    data = []
    labels = []
    train_y = []
    # grab the image paths and randomly shuffle them
    
    labels = os.listdir(path)
    #labels.remove('.DS_Store')
    labels.remove('.DS_Store')
    imagePaths = []
    print(labels)
    for i, label in enumerate(labels):
        path_file=os.path.join(path, label)
        #path_file = os.path.join(path,'label')
        path_files = os.path.join(path_file,p)
        imagePath = os.listdir(path_files)
        #print(path_files)
        #imagePath.remove('.DS_Store')
        for img in imagePath:
            if img == '.DS_Store':
                print('wrong')
            else:
                image = os.path.join(path_files,img)
                imagePaths.append(image)
            #print(path_files)
    #print(imagePaths)
    random.seed(42)
    random.shuffle(imagePaths)
    
    return imagePaths

In [4]:
part='train'
trainlist = generate_file_list(path='save/',p=part) 


[INFO] loading images...
['ABSOLUTELY', 'ABOUT', 'ABUSE']
wrong
wrong


# write

In [5]:
fileObject = open('filelist.txt', 'w')
for ip in trainlist:
    fileObject.write(ip)
    fileObject.write('\n')
fileObject.close()

In [6]:
train_list = []
fd = open( "filelist.txt", "r" )

for line in fd.readlines():
    train_list.append(line.rstrip('\n'))
    #print(line)
#print(train_list)

In [7]:
def load_data(path,p):
    print("[INFO] loading images...")
    data = []
    labels = []
    train_y = []
    # grab the image paths and randomly shuffle them
    
    labels = os.listdir(path)
    labels.remove('.DS_Store')
    imagePaths = []
    print(labels)
    for i, label in enumerate(labels):
        path_file=os.path.join(path, label)
        #path_file = os.path.join(path,'label')
        path_files = os.path.join(path_file,p)
        imagePath = os.listdir(path_files)
        #print(path_files)
        #imagePath.remove('.DS_Store')
        for img in imagePath:
            if img == '.DS_Store':
                print('wrong')
            else:
                image = os.path.join(path_files,img)
                imagePaths.append(image)
            #print(path_files)
    #print(imagePaths)
    random.seed(42)
    random.shuffle(imagePaths)
    #print(imagePaths)
    # loop over the input images
    for imagePath in imagePaths:
        image_path = sorted(list(paths.list_images(imagePath)))
        images=[]
        for img in image_path:
            
            # load the image, pre-process it, and store it in the data list
            image = cv2.imread(img)
            #image = cv2.resize(image, (norm_size, norm_size))
            #image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
            #image = cv2.resize(image, (norm_size, norm_size))
            image = img_to_array(image)
            #print(image)
            #image = np.array(image, dtype="float") / 255.0
            #print(image.shape)
            image = np.reshape(image,(120,120,3))
            images.append(image)
        
        images=np.array(images,dtype="float")
        if(len(images)==0):
            print(imagePath)
            break
        #images=np.reshape(images,(120,120,29))
        #print(images.shape)
        #print(images.shape)
        data.append(images)

        # extract the class label from the image path and update the
        # labels list
        #label = int(imagePath.split(os.path.sep)[-2])
        basename = os.path.basename(imagePath)
        label = basename.split('_')
        train_y.append(label[0])
        #print(len(labels))
    #data = np.array(data, dtype="float") / 255.0
    data = np.array(data, dtype="float")
    #print(data.shape)

    # scale the raw pixel intensities to the range [0, 1]
    train_y = np.array(train_y)
    print(train_y.shape)
    # 整数编码
    integer_result = label_encoder.transform(train_y)
    # One-Hot编码
    #one_hot_encoder = OneHotEncoder()
# One-Hot编码将分类值映射到整数值再表示成二进制向量
    integer_result1 = integer_result.reshape(len(integer_result), 1)
    y_label = one_hot_encoder.transform(integer_result1)

    #labels = np.array(labels)

    # convert the labels from integers to vectors
    #labels = to_categorical(labels, num_classes=CLASS_NUM)                         
    return data,y_label

In [8]:
part='val'
valX,valY = load_data(path='save/',p=part) 

part='test'
testX,testY = load_data(path='save/',p=part) 

[INFO] loading images...
['ABSOLUTELY', 'ABOUT', 'ABUSE']
(150,)
[INFO] loading images...
['ABSOLUTELY', 'ABOUT', 'ABUSE']
wrong
wrong
(150,)


In [9]:
part='train'
trainX,trainY = load_data(path='save/',p=part)
train_Y=trainY.toarray()

[INFO] loading images...
['ABSOLUTELY', 'ABOUT', 'ABUSE']
wrong
wrong
(2959,)


In [10]:
val_Y=valY.toarray()
test_Y = testY.toarray()

In [2]:
from keras.models import Sequential,Model
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D,Conv3D
from keras.layers.convolutional import MaxPooling2D,MaxPooling3D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import LSTM,Input
from keras import backend as K
from keras.layers.wrappers import TimeDistributed, Bidirectional
#from keras.layers.extra import TimeDistributedConvolution2D, TimeDistributedMaxPooling2D, TimeDistributedFlatten

In [3]:
class_num = 3

In [4]:
#inputShape = (29,120,120,3)
input1 = Input(shape = (29,120,120,3))

conv3d = Conv3D(64,(5,7,7),padding = 'same',
                strides = (1,2,2), activation='relu')(input1)
batch_norm1 = BatchNormalization()(conv3d)
maxpool3d = MaxPooling3D(pool_size=(1,2,2),padding='same')(batch_norm1)

time_conv2d = TimeDistributed(Conv2D(128,(3,3),activation = 'relu'))(maxpool3d)
batch_norm2 = BatchNormalization()(time_conv2d)
time_maxpool2d = TimeDistributed(MaxPooling2D(2,2))(batch_norm2)

time_conv2d = TimeDistributed(Conv2D(256,(3,3),activation = 'relu'))(time_maxpool2d)
batch_norm3 = BatchNormalization()(time_conv2d)
time_maxpool2d = TimeDistributed(MaxPooling2D(2,2))(batch_norm3)

time_flatten = TimeDistributed(Flatten())(time_maxpool2d)

bilstm1 = Bidirectional(LSTM(128,return_sequences=True))(time_flatten)
bilstm2 = Bidirectional(LSTM(128,return_sequences=True))(bilstm1)

flatten = Flatten()(bilstm2)
dense1 = Dense(1024,activation='relu')(flatten)
batch_norm4 = BatchNormalization()(dense1)
dense2 = Dense(class_num,activation = 'softmax')(batch_norm4)

In [5]:
model = Model(inputs=[input1], 
              outputs=[dense2])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 29, 120, 120, 3)   0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 29, 60, 60, 64)    47104     
_________________________________________________________________
batch_normalization_1 (Batch (None, 29, 60, 60, 64)    256       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 29, 30, 30, 64)    0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 29, 28, 28, 128)   73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 29, 28, 28, 128)   512       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 29, 14, 14, 128)   0         
__________

In [6]:
json_string = model.to_json()
with open('mlp_model.json','w') as of:
    of.write(json_string)

In [7]:
from keras.utils import plot_model
plot_model(model, to_file='model_video.png')

In [15]:
# EPOCHS = 3
# INIT_LR = 0.1
# BS = 16
# IMAGE_DIMS = (120,120,29)

# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range = 10, width_shift_range=0.1,
	height_shift_range=0.1, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")
# aug = ImageDataGenerator()

In [16]:
print("[INFO] compiling model...")
# model = SmallerVGGNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],
# 	depth=IMAGE_DIMS[2], classes=6)
#opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=sgd,
	metrics=["accuracy"])

[INFO] compiling model...


In [ ]:
model.fit(trainX, train_Y, epochs=1, batch_size=32, 
          verbose=1, validation_data=(valX, val_Y))

Train on 2959 samples, validate on 150 samples
Epoch 1/1
  32/2959 [..............................] - ETA: 25:00:02 - loss: 1.8230 - acc: 0.2188

In [ ]:
# print("[INFO] training network...")
# H = model.fit_generator(
# 	aug.flow(trainX, train_Y, batch_size=32),
# 	validation_data=(testX, test_Y),
# 	steps_per_epoch=len(trainX) // 32,
# 	epochs=EPOCHS, verbose=1)

In [ ]:
model.evaluate(testX,test_Y)